# Temporal Memory Tests

**Goal:**  Verify that the temporal memory can learn sequences of
* Different lengths
* Varying overlaps

**Method:** Encode a circular, sequential pattern.  Verify that the TP can learn the pattern and reproduce it accurately.  Vary the length and overlap of the pattern.  Determine any limitations of the TP w.r.t. learning circular, sequential patterns.


In [39]:
import calendar
import datetime
import numpy
import os.path
import pickle
from random import randrange, random, shuffle
import sys
import time

import nupic
from nupic.encoders import ScalarEncoder, MultiEncoder
from nupic.bindings.algorithms import SpatialPooler as SP
from nupic.research.TP10X2 import TP10X2 as TP

<img src="https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSKjXKRmpH8060mV6gRfZNqXJp2ozRiiLQ1FI7yJgVAZQpnp57J" width="40px" height="40px"align="left"/>
<br/>  Import all the good stuff

In [176]:
A = [1] * 30 + [0] * 60
B = [0] * 30 + [1] * 30 + [0] * 30
C = [0] * 60 + [1] * 30
n = 10
w = 5
inputs = [[0] * (i*w) + [1] * w + [0] * ((n-i-1)*w) for i in range(0,n)]

enc = ScalarEncoder(w=5, minval=0, maxval=10, radius=1.25, periodic=True, name="encoder", forced=True)
for d in range(0, 10):
    print str(enc.encode(d))
inputs = [enc.encode(i) for i in range(10)]
print len(inputs[0])

[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1]
[0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
[0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0
 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0
 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 0]
40


In [190]:
tp = TP(numberOfCols=40, cellsPerColumn=7,
        initialPerm=0.5, connectedPerm=0.5,
        minThreshold=10, newSynapseCount=10,
        permanenceInc=0.1, permanenceDec=0.01,
        activationThreshold=1,
        globalDecay=0, burnIn=1,
        checkSynapseConsistency=False,
        pamLength=7)

In [194]:
input_array = numpy.zeros(40, dtype="int32")
tp.reset()
for i, pattern in enumerate(inputs):
    input_array[:] = pattern
    tp.compute(input_array, enableLearn=True, computeInfOutput=True)
    tp.printStates()


Inference Active state
0000000000 0000000000 0000000000 0000000000  1110000000 0000000000 0000000000 0000000011 
0000000000 0000000000 0000000000 0000000000  0000000000 0000000000 0000000000 0000000000 
0000000000 0000000000 0000000000 0000000000  0000000000 0000000000 0000000000 0000000000 
0000000000 0000000000 0000000000 0000000000  0000000000 0000000000 0000000000 0000000000 
0000000000 0000000000 0000000000 0000000000  0000000000 0000000000 0000000000 0000000000 
0000000000 0000000000 0000000000 0000000000  0000000000 0000000000 0000000000 0000000000 
0000000000 0000000000 0000000000 0000000000  0000000000 0000000000 0000000000 0000000000 
Inference Predicted state
0000000000 0000000000 0000000000 0000000000  0000000000 0000000000 0000000000 0000000000 
0000000000 0000000000 0000000000 0000000000  0011000000 0000000000 0000000000 0000000000 
0000000000 0000000000 0000000000 0000000000  0000000000 0000000000 0000000000 0000000000 
0000000000 0000000000 0000000000 0000000000  00000

In [104]:
w = 5
n = 10
str([[0] * (i*w) + [1] * w + [0] * ((n-i-1)*w) for i in range(0,n)]).replace('\n', '')


'[[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0